In [1]:
import json
import os

In [2]:
import pandas as pd
import networkx as nx

In [3]:
from iac_sketch import data, etl, sketch, transform
from iac_sketch.extract import extract_python, extract_yaml
from iac_sketch import system_tests

In [4]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(extract_yaml)
importlib.reload(extract_python)
importlib.reload(etl)
importlib.reload(transform)
importlib.reload(system_tests)
importlib.reload(sketch)

<module 'iac_sketch.sketch' from '/Users/zhafen/repos/iac-sketch/iac_sketch/sketch.py'>

In [5]:
architect = sketch.Architect(
    root_dir="./test_data/healthcare_example",
    filename_patterns=[
        "./manifest/**/*.yaml",
        "./ra_lib/**/*.py",
    ],
)
registry = architect.perform_registry_etl()
tests, test_results = architect.validate_registry(
    min_priority=0.7,
    allowed_infrastructure=["research_analytics_infrastructure"],
)

<unknown>:368: SyntaxWarning: invalid escape sequence '\s'
<unknown>:408: SyntaxWarning: invalid escape sequence '\R'
<unknown>:357: SyntaxWarning: invalid escape sequence '\R'
<unknown>:359: SyntaxWarning: invalid escape sequence '\e'
<unknown>:360: SyntaxWarning: invalid escape sequence '\e'
<unknown>:361: SyntaxWarning: invalid escape sequence '\e'
<unknown>:362: SyntaxWarning: invalid escape sequence '\e'


entity: ../../../iac_sketch/system_tests.test_designed
requirement: fully_designed
priority: 0.9
description:
    Each [requirement] entity has at least one [satisfies] entity or one [child]
    entity.
test_passed: True
--------------------------------------------------------------------------------

entity: ../../../iac_sketch/system_tests.test_defined
requirement: fully_defined
priority: 0.8
description:
    All [compdef] entities are defined and valid.
test_passed: True
--------------------------------------------------------------------------------

entity: ../../../iac_sketch/system_tests.test_implemented
requirement: fully_implemented
priority: 0.7
description:
    Meets the same criteria as "fully_designed" and all [satisfies] entities
    either have a test that passes when executed or have a status of "in
    production".
test_passed: False
failed_components:


,entity,requirement.comp_key,requirement.priority,requirement.value,link.source,link.target,link.link_type,link.value,status.comp_key,status.value,test.comp_key,test.value
0,can_accept_and_process_request,1,1.0,research_analytics_infrastructure,intake_request_workflow,can_accept_and_process_request,satisfies,NaN,2,in development,NaN,NaN
1,can_accept_and_process_request,1,1.0,research_analytics_infrastructure,intake_request_workflow,can_accept_and_process_request,satisfies,NaN,5,in development,NaN,NaN
2,can_approve_analyst_role,1,0.9,NaN,analyst_skills_assessment,can_approve_analyst_role,satisfies,NaN,2,new,NaN,NaN
3,can_approve_power_user,0,0.9,NaN,power_user_approval_process,can_approve_power_user,satisfies,NaN,1,in development,NaN,NaN
6,can_audit_data_access_and_usage,0,1.0,research_analytics_infrastructure,compliance_reports_suite,can_audit_data_access_and_usage,satisfies,NaN,2,new,NaN,NaN
8,can_build_reports,0,1.0,research_analytics_infrastructure,build_report_workflow,can_build_reports,satisfies,NaN,4,in development,NaN,NaN
13,can_complete_requests_with_analysts,1,1.0,NaN,can_support_and_manage_analysts,can_complete_requests_with_analysts,satisfies,NaN,NaN,NaN,NaN,NaN
14,can_complete_requests_with_analysts,1,1.0,NaN,cloud_framework_is_usable,can_complete_requests_with_analysts,satisfies,NaN,NaN,NaN,NaN,NaN
15,can_control_report_access,0,1.0,research_analytics_infrastructure,set_report_access,can_control_report_access,satisfies,NaN,4,in development,NaN,NaN
21,can_deliver_data_for_a_given_request,0,1.0,research_analytics_infrastructure,request_workflow,can_deliver_data_for_a_given_request,satisfies,NaN,3,in development,NaN,NaN


--------------------------------------------------------------------------------



In [6]:
# Select entities to export
entities = (
    registry.view("entity_source")
    .query("source == 'user'")
    .index.get_level_values("entity")
    .unique()
)
entities

Index(['accept_intake_form', 'access_package',
       'access_pbi_report_using_direct_link',
       'access_pbi_report_via_data_portal', 'analyst_laptop',
       'analyst_skills_assessment', 'apply_security_groups_to_reports',
       'assign_analyst', 'azdo_work_item', 'basic_report_test',
       ...
       'turn_on_pbi_refresh', 'updated_report_new_analyst_test',
       'updated_report_test', 'user_principal_name',
       'variable_naming_conventions', 'work_on_report', 'worklog',
       'write_report_to_catalog', 'write_report_to_sharepoint',
       'write_report_to_volume'],
      dtype='object', name='entity', length=963)

In [7]:
# Get component instances for those entities
# and add the component types themselves to the list of entities.
# Repeat until no new entities are found
compinsts = registry.view("compinst")
n_current = len(entities)
n_new = n_current
while n_new > 0:
    new_entities = compinsts.loc[entities, "component_type"].unique()
    entities = entities.union(new_entities)
    n_new = len(entities) - n_current
    n_current = len(entities)
entities

Index(['Call', 'ClassDef', 'FunctionDef', 'Import', 'ImportFrom', 'Module',
       'accept_intake_form', 'access_package',
       'access_pbi_report_using_direct_link',
       'access_pbi_report_via_data_portal',
       ...
       'updated_report_new_analyst_test', 'updated_report_test', 'url',
       'user_principal_name', 'variable_naming_conventions', 'work_on_report',
       'worklog', 'write_report_to_catalog', 'write_report_to_sharepoint',
       'write_report_to_volume'],
      dtype='object', name='entity', length=1002)

In [8]:
# Get nodes data
entity_data = []
skip_types = ["compdef"]
for entity in entities:
    entity_data_i = registry.view_entity(
        entity,
        output_yaml=False,
        print_output=False,
    )

    entity_data_i = {k: v for k, v in entity_data_i.items() if k not in skip_types}

    entity_data_str_i = json.dumps(entity_data_i)
    entity_data.append((entity, entity_data_str_i))

In [9]:
# Get edges data
links = registry.view("link").query("source.isin(@entities) & target.isin(@entities)")

In [10]:
# Build graph
graph = nx.from_pandas_edgelist(
    links,
    source="source",
    target="target",
    edge_key="link_type",
    create_using=nx.MultiDiGraph,
)
graph.add_nodes_from(entity_data)

In [12]:
# Export to graph ML
output_fp = os.path.join(architect.root_dir, "generated/healthcare_example.graphml")
nx.write_graphml(graph, output_fp)